# Mekanisme Perhatian dan Transformer

Salah satu kelemahan utama jaringan berulang (recurrent networks) adalah semua kata dalam sebuah urutan memiliki dampak yang sama terhadap hasil. Hal ini menyebabkan performa yang kurang optimal pada model encoder-decoder LSTM standar untuk tugas sequence-to-sequence, seperti Named Entity Recognition dan Machine Translation. Pada kenyataannya, kata-kata tertentu dalam urutan input sering kali memiliki pengaruh lebih besar terhadap output berurutan dibandingkan yang lain.

Pertimbangkan model sequence-to-sequence, seperti penerjemahan mesin. Model ini diimplementasikan dengan dua jaringan berulang, di mana satu jaringan (**encoder**) akan merangkum urutan input menjadi state tersembunyi, dan jaringan lainnya, **decoder**, akan mengurai state tersembunyi ini menjadi hasil terjemahan. Masalah dengan pendekatan ini adalah state akhir dari jaringan akan kesulitan mengingat awal kalimat, sehingga menyebabkan kualitas model yang buruk pada kalimat panjang.

**Mekanisme Perhatian** menyediakan cara untuk memberikan bobot pada dampak kontekstual dari setiap vektor input terhadap setiap prediksi output dari RNN. Cara ini diimplementasikan dengan menciptakan jalur pintas antara state antara dari RNN input dan RNN output. Dengan cara ini, saat menghasilkan simbol output $y_t$, kita akan mempertimbangkan semua state tersembunyi input $h_i$, dengan koefisien bobot yang berbeda $\alpha_{t,i}$. 

![Gambar yang menunjukkan model encoder/decoder dengan lapisan perhatian aditif](../../../../../translated_images/id/encoder-decoder-attention.7a726296894fb567.webp)
*Model encoder-decoder dengan mekanisme perhatian aditif dalam [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), dikutip dari [blog ini](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matriks perhatian $\{\alpha_{i,j}\}$ akan merepresentasikan sejauh mana kata-kata tertentu dalam input berperan dalam menghasilkan kata tertentu dalam urutan output. Berikut adalah contoh matriks seperti itu:

![Gambar yang menunjukkan contoh alignment yang ditemukan oleh RNNsearch-50, diambil dari Bahdanau - arviz.org](../../../../../translated_images/id/bahdanau-fig3.09ba2d37f202a6af.webp)

*Gambar diambil dari [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Mekanisme perhatian bertanggung jawab atas banyak pencapaian terkini atau hampir terkini dalam pemrosesan bahasa alami. Namun, menambahkan perhatian secara signifikan meningkatkan jumlah parameter model, yang menyebabkan masalah skala pada RNN. Salah satu kendala utama dalam skala RNN adalah sifat berulang dari model yang membuatnya sulit untuk melakukan pelatihan secara batch dan paralel. Dalam RNN, setiap elemen dari urutan harus diproses secara berurutan, yang berarti tidak dapat dengan mudah diparalelkan.

Adopsi mekanisme perhatian yang dikombinasikan dengan kendala ini mengarah pada terciptanya Model Transformer yang kini menjadi State of the Art, seperti BERT hingga OpenGPT3, yang kita kenal dan gunakan saat ini.

## Model Transformer

Alih-alih meneruskan konteks dari setiap prediksi sebelumnya ke langkah evaluasi berikutnya, **model transformer** menggunakan **positional encodings** dan perhatian untuk menangkap konteks dari input tertentu dalam jendela teks yang diberikan. Gambar di bawah ini menunjukkan bagaimana positional encodings dengan perhatian dapat menangkap konteks dalam jendela tertentu.

![GIF animasi yang menunjukkan bagaimana evaluasi dilakukan dalam model transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Karena setiap posisi input dipetakan secara independen ke setiap posisi output, transformer dapat melakukan paralelisasi lebih baik dibandingkan RNN, yang memungkinkan model bahasa yang jauh lebih besar dan lebih ekspresif. Setiap kepala perhatian dapat digunakan untuk mempelajari hubungan yang berbeda antara kata-kata yang meningkatkan tugas Pemrosesan Bahasa Alami di hilir.

**BERT** (Bidirectional Encoder Representations from Transformers) adalah jaringan transformer multi-layer yang sangat besar dengan 12 lapisan untuk *BERT-base*, dan 24 untuk *BERT-large*. Model ini pertama kali dilatih pada korpus teks besar (WikiPedia + buku) menggunakan pelatihan tanpa pengawasan (memprediksi kata yang disembunyikan dalam sebuah kalimat). Selama pelatihan awal, model menyerap tingkat pemahaman bahasa yang signifikan yang kemudian dapat dimanfaatkan dengan dataset lain menggunakan fine tuning. Proses ini disebut **transfer learning**. 

![gambar dari http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/id/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Ada banyak variasi arsitektur Transformer termasuk BERT, DistilBERT, BigBird, OpenGPT3, dan lainnya yang dapat disesuaikan. Paket [HuggingFace](https://github.com/huggingface/) menyediakan repositori untuk melatih banyak arsitektur ini dengan PyTorch. 

## Menggunakan BERT untuk klasifikasi teks

Mari kita lihat bagaimana kita dapat menggunakan model BERT yang telah dilatih sebelumnya untuk menyelesaikan tugas tradisional kita: klasifikasi urutan. Kita akan mengklasifikasikan dataset AG News asli kita.

Pertama, mari kita muat pustaka HuggingFace dan dataset kita:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Karena kita akan menggunakan model BERT yang telah dilatih sebelumnya, kita perlu menggunakan tokenizer tertentu. Pertama, kita akan memuat tokenizer yang terkait dengan model BERT yang telah dilatih sebelumnya.

Perpustakaan HuggingFace memiliki repositori model yang telah dilatih sebelumnya, yang dapat Anda gunakan hanya dengan menyebutkan nama model tersebut sebagai argumen untuk fungsi `from_pretrained`. Semua file biner yang diperlukan untuk model akan otomatis diunduh.

Namun, pada waktu tertentu Anda mungkin perlu memuat model Anda sendiri. Dalam kasus ini, Anda dapat menentukan direktori yang berisi semua file terkait, termasuk parameter untuk tokenizer, file `config.json` dengan parameter model, bobot biner, dan sebagainya.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objek `tokenizer` berisi fungsi `encode` yang dapat langsung digunakan untuk mengenkripsi teks:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Kemudian, mari kita buat iterator yang akan kita gunakan selama pelatihan untuk mengakses data. Karena BERT menggunakan fungsi encoding-nya sendiri, kita perlu mendefinisikan fungsi padding yang mirip dengan `padify` yang telah kita definisikan sebelumnya:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Dalam kasus kami, kami akan menggunakan model BERT yang telah dilatih sebelumnya yang disebut `bert-base-uncased`. Mari kita muat model menggunakan paket `BertForSequenceClassfication`. Ini memastikan bahwa model kami sudah memiliki arsitektur yang diperlukan untuk klasifikasi, termasuk classifier akhir. Anda akan melihat pesan peringatan yang menyatakan bahwa bobot classifier akhir belum diinisialisasi, dan model akan memerlukan pelatihan awal - itu sepenuhnya baik-baik saja, karena itulah yang akan kita lakukan!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Sekarang kita siap untuk memulai pelatihan! Karena BERT sudah dilatih sebelumnya, kita ingin memulai dengan tingkat pembelajaran yang cukup kecil agar tidak merusak bobot awal.

Semua pekerjaan berat dilakukan oleh model `BertForSequenceClassification`. Ketika kita memanggil model pada data pelatihan, model ini mengembalikan baik nilai kerugian (loss) maupun output jaringan untuk minibatch input. Kita menggunakan nilai kerugian untuk optimasi parameter (`loss.backward()` melakukan proses mundur), dan `out` untuk menghitung akurasi pelatihan dengan membandingkan label yang diperoleh `labs` (dihitung menggunakan `argmax`) dengan label yang diharapkan `labels`.

Untuk mengontrol prosesnya, kita mengakumulasi nilai kerugian dan akurasi selama beberapa iterasi, lalu mencetaknya setiap siklus pelatihan `report_freq`.

Pelatihan ini kemungkinan akan memakan waktu cukup lama, jadi kita membatasi jumlah iterasi.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Anda dapat melihat (terutama jika Anda meningkatkan jumlah iterasi dan menunggu cukup lama) bahwa klasifikasi menggunakan BERT memberikan akurasi yang cukup baik! Hal ini karena BERT sudah memahami struktur bahasa dengan cukup baik, dan kita hanya perlu menyempurnakan pengklasifikasi akhirnya. Namun, karena BERT adalah model yang besar, seluruh proses pelatihan memakan waktu lama dan membutuhkan daya komputasi yang signifikan! (GPU, dan sebaiknya lebih dari satu).

> **Catatan:** Dalam contoh kita, kita menggunakan salah satu model BERT pra-latih terkecil. Ada model yang lebih besar yang kemungkinan memberikan hasil yang lebih baik.


## Mengevaluasi kinerja model

Sekarang kita dapat mengevaluasi kinerja model kita pada dataset pengujian. Proses evaluasi sangat mirip dengan proses pelatihan, tetapi kita tidak boleh lupa untuk mengubah model ke mode evaluasi dengan memanggil `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Poin Penting

Dalam unit ini, kita telah melihat betapa mudahnya mengambil model bahasa yang sudah dilatih sebelumnya dari pustaka **transformers** dan menyesuaikannya untuk tugas klasifikasi teks kita. Demikian pula, model BERT dapat digunakan untuk ekstraksi entitas, menjawab pertanyaan, dan tugas NLP lainnya.

Model transformer mewakili teknologi terkini dalam NLP, dan dalam sebagian besar kasus, ini seharusnya menjadi solusi pertama yang Anda coba saat mengimplementasikan solusi NLP kustom. Namun, memahami prinsip dasar dari jaringan saraf berulang yang dibahas dalam modul ini sangatlah penting jika Anda ingin membangun model saraf yang lebih canggih.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
